In [5]:
from huggingface_hub import hf_hub_download

## Generate Flie List

In [3]:
def calculate_total_days(start_year, start_month, end_year, end_month):
    # Define the number of days in each month
    days_in_month = {
        1: 31,
        2: 28,  
        3: 31,
        4: 30,
        5: 31,
        6: 30,
        7: 31,
        8: 31,
        9: 30,
        10: 31,
        11: 30,
        12: 31,
    }

    # Calculate the number of full years
    full_years = end_year - start_year-1

    # Calculate the number of days in full years
    total_days = full_years * 365  # Ignoring leap years

    # Add days for each month from start_month to the end of the last year
    for month in range(start_month, 13):
        total_days += days_in_month[month]

    # Add days for each month from the first month of the end year to end_month
    for month in range(1, end_month + 1):
        total_days += days_in_month[month]

    return total_days


def generate_file_list(time_period):
    current_time = 0
    current_day = 1
    current_month = int(time_period[0].split("-")[1])
    current_year = int(time_period[0].split("-")[0])
    end_month = int(time_period[1].split("-")[1])
    end_year = int(time_period[1].split("-")[0])
    start_date = (current_year, current_month) 
    end_date = (end_year, end_month)   
    total_days = calculate_total_days(start_date[0], start_date[1], end_date[0], end_date[1])
    end_time = 86400 * total_days # Adjust as needed for the number of days in the month

    # Define the time step (in seconds)
    time_step = 1200

    days_in_month = {
        1: 31,
        2: 28, 
        3: 31,
        4: 30,
        5: 31,
        6: 30,
        7: 31,
        8: 31,
        9: 30,
        10: 31,
        11: 30,
        12: 31,
    }

    file_list = []
    folder_list = []
    target_list = []
    current_target_file_list = []
    current_file_list = []
    current_folder = []

    # Loop through the time range and download the files
    while current_time <= end_time:
        # Calculate the day and time in day
        day = current_day
        time_in_day = current_time % 86400

        # Generate the file name based on the current date and time
        sub_folder = f"train/{current_year:04d}-{current_month:02d}"
        file_name = f"E3SM-MMF.mli.{current_year:04d}-{current_month:02d}-{day:02d}-{time_in_day:05d}.nc"
        target_file_name = f"E3SM-MMF.mlo.{current_year:04d}-{current_month:02d}-{day:02d}-{time_in_day:05d}.nc"

        current_target_file_list.append(target_file_name)
        current_file_list.append(file_name)
        current_folder.append(sub_folder)

        # Update the current_time and date
        current_time += time_step
        if time_in_day + time_step >= 86400:
            current_day += 1
            if current_day > days_in_month[current_month]:
                file_list.append(current_file_list)
                folder_list.append(current_folder)
                target_list.append(current_target_file_list)
                current_target_file_list = []
                current_file_list = []
                current_folder = []
                current_month += 1
                current_day = 1
                if current_month > 12:
                    current_year += 1
                    current_month = 1    
    return file_list, target_list, folder_list

## Downloading files

In [6]:
def download_nc(time_period, local_dir):
    file_list = generate_file_list(time_period)[0]
    target_list = generate_file_list(time_period)[1]
    folder_list = generate_file_list(time_period)[2]
    repo_id = "LEAP/ClimSim_low-res"
    
    #Download file list
    for i in range(len(file_list)):
        for j in range(len(file_list[i])):
            local_filepath = hf_hub_download(repo_id=repo_id, filename=file_list[i][j], subfolder = folder_list[i][j], repo_type="dataset", local_dir = local_dir, local_dir_use_symlinks = 'auto')

    #Download target file list
    for i in range(len(target_list)):
        for j in range(len(target_list[i])):
            local_filepath = hf_hub_download(repo_id=repo_id, filename=target_list[i][j], subfolder = folder_list[i][j], repo_type="dataset", local_dir = local_dir, local_dir_use_symlinks = 'auto')